In [1]:
import os
import os.path as op
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline
import seaborn as sns
sns.set_theme(style="darkgrid", color_codes=True)
sns.set(font_scale=1.35, style="ticks") #set styling preferences
import statsmodels.api as sm
from scipy import stats
import math
from math import pi
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from scipy.spatial.distance import cdist
from scipy.cluster.vq import kmeans2,vq, whiten

In [2]:
# Show all columns and rows
pd.options.display.max_columns = None
pd.options.display.max_rows = None

#### Baseline

In [3]:
%%time
baseline2018 = "https://beam-core-act.s3.amazonaws.com/deepDive/CleanData/SanFrancisco/Baseline/"
sfbase = pd.read_csv(baseline2018 + 'sf_2018_base_8_22_22.csv')

<timed exec>:2: DtypeWarning: Columns (42) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: total: 1min 12s
Wall time: 4min 11s


In [4]:
# Showing the entire number in dataframe
pd.set_option('float_format', '{:f}'.format)

In [5]:
#describe = sfbase.describe()

In [6]:
#describe.to_csv('C:/Shared-Work/Data/CleanData/ASIM_BEAM_Merged/describe.csv')

#### Ridehail Price-Fleetsize Scenarios

In [7]:
%%time
rh_price_path = "https://beam-core-act.s3.amazonaws.com/deepDive/CleanData/SanFrancisco/Ridehail_Price_Fleetsize/"
sf_rh_price_2 = pd.read_csv(rh_price_path + 'sf_2018_rh_price_2_flsz_0p5.csv')
sf_rh_price_1p5 = pd.read_csv(rh_price_path + 'sf_2018_rh_price_1p5_flsz_0p5.csv')
sf_rh_price_0p5 = pd.read_csv(rh_price_path + 'sf_2018_rh_price_0p5_flsz_0p5.csv')

<timed exec>:2: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
<timed exec>:3: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: total: 3min 58s
Wall time: 13min 34s


In [8]:
sfbase.shape

(2468267, 257)

In [10]:
sf_rh_price_1p5.shape

(2468149, 258)

In [ ]:
#sf_rh_flsz_175 = sf_rh_flsz_175.drop(sf_rh_flsz_175.columns[245:625], axis=1)

In [ ]:
#sf_rh_flsz_175 = sf_rh_flsz_175.drop(sf_rh_flsz_175.columns[235:245], axis=1)

In [14]:
#sf_rh_flsz_175 = sf_rh_flsz_175.drop(sf_rh_flsz_175.columns[233:235], axis=1)

In [8]:
frames = [sf_rh_price_2, sf_rh_price_1p5, sf_rh_price_0p5, sfbase]
sf_stacked = pd.concat(frames)

In [12]:
sf_stacked = sf_stacked.drop(['BlockGroupEnd','BlockGroupStart','has_school_kid','has_preschool_kid','has_retiree','has_non_worker','adult','hispanic','relate',
                             'pstudent','pemploy','ptype','p_hispanic','age_16_p','age_16_to_19','work_zone_id',
                             'race_id','school_zone_id','person_age','MAR','hours','hispanic.1','student',
                             'worker','stop_frequency','composition','tdd','number_of_participants','tour_type_count',
                             'tour_type_num','tour_num','tour_count', 'outbound', 'trip_num','num_under16_not_at_school',
                             'family','non_family','num_young_adults','num_college_age','num_children_16_to_17',
                              'num_children_5_to_15','num_young_children'], axis=1)

In [13]:
sf_stacked = sf_stacked.drop(['vehicleIds_estimate','destination_x','origin_x','depart',
                             'origin_y', 'destination_y','parent_tour_id','atwork_subtour_frequency','household_id_y',
                              'TAZ_x','home_x','home_y','male','female', 'has_driving_kid','has_full_time','has_part_time',
                              'has_university','student_is_employed', 'nonstudent_to_school', 'is_student','is_gradeschool',
                              'is_gradeschool','is_highschool','is_university','school_segment','is_worker','home_taz',  
                             'school_taz','distance_to_school','roundtrip_auto_time_to_school','workplace_taz',
                              'distance_to_work','workplace_in_cbd', 'work_taz_area_type', 'hh_race_of_head',
                             'roundtrip_auto_time_to_work','work_auto_savings_ratio','cdap_activity', 'travel_active', 
                              'under16_not_at_school','has_preschool_kid_at_home', 'block_id','gt2','hispanic_head',
                             'has_school_kid_at_home','work_and_school_and_worker', 'age_of_head', 'race_of_head',
                              'work_and_school_and_student', 'VEHICL','hh_children','hh_age_of_head','num_workers',
                              'gt55','seniors', 'recent_mover', 'hh_workers', 'hispanic_status_of_head', 'hh_seniors', 
                             'hh_type', 'TAZ_y', 'HHT', 'sample_rate', 'chunk_id', 'income_segment', 'num_non_workers', 
                             'num_drivers', 'num_adults', 'num_children', 'hh_work_auto_savings_ratio', 'num_travel_active', 
                             'num_travel_active_adults','num_travel_active_preschoolers', 'num_travel_active_children', 
                              'num_travel_active_non_preschoolers', 'participates_in_jtf_model', 'joint_tour_frequency',
                              'num_hh_joint_tours'], axis=1)

In [14]:
sf_stacked = sf_stacked.drop(['serialno'], axis=1)

#### Stacked File

In [15]:
#stacked_tr_fr_path = "https://beam-core-act.s3.amazonaws.com/deepDive/CleanData/SanFrancisco/Stacked/"
#sf_stacked_tr_fr = pd.read_csv(stacked_tr_fr_path + 'sf_2018_stacked_tr_fr.csv')

In [9]:
sf_stacked['lever_position_fleetsize'] = np.where(sf_stacked['lever_position'] == 1, 1 , sf_stacked['lever_position_fleetsize'])

In [10]:
sf_stacked['lever_position_price'] = np.where(sf_stacked['lever_position'] == 1, 1 , sf_stacked['lever_position_price'])

In [21]:
sf_stacked = sf_stacked.sort_values(by=['IDMerged', 'tripIndex', 'lever_position_price', 'lever_position_fleetsize']).reset_index(drop=True)

In [12]:
# shift column 'person' to first position
first_column = sf_stacked.pop('scenario')
second_column = sf_stacked.pop('lever')
third_column = sf_stacked.pop('lever_position_fleetsize')
fourth_column = sf_stacked.pop('lever_position_price')
fifth_column = sf_stacked.pop('logsum_trip_mode_AS_trips')
# insert column using insert(position,column_name,first_column) function
sf_stacked.insert(2, 'scenario', first_column)
sf_stacked.insert(3, 'lever', second_column)
sf_stacked.insert(4, 'lever_position_fleetsize', third_column)
sf_stacked.insert(5, 'lever_position_price', fourth_column)
sf_stacked.insert(6, 'logsum_trip_mode_AS_trips', fifth_column)

C:\Users\nazanin\AppData\Local\Temp\2\ipykernel_8860\3311588988.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sf_stacked.insert(2, 'scenario', first_column)
C:\Users\nazanin\AppData\Local\Temp\2\ipykernel_8860\3311588988.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sf_stacked.insert(3, 'lever', second_column)
C:\Users\nazanin\AppData\Local\Temp\2\ipykernel_8860\3311588988.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perfor

In [20]:
#sf_stacked_tr_fr['logsum_relative_to_baseline'] = sf_stacked_tr_fr.groupby(['IDMerged','tripIndex'])['logsum_trip_mode_AS_trips'].apply(lambda x: x.diff())

In [21]:
#sf_stacked_tr_fr['logsum_relative_to_baseline'] = sf_stacked_tr_fr['logsum_trip_mode_AS_trips'] - sf_stacked_tr_fr.loc[sf_stacked_tr_fr.groupby(['IDMerged','tripIndex'])['lever_position'].transform('idxmin'), 'logsum_trip_mode_AS_trips'].values

In [13]:
%%time
sf_stacked['logsum_relative_to_baseline'] = sf_stacked['logsum_trip_mode_AS_trips'] - (sf_stacked['logsum_trip_mode_AS_trips'].where(sf_stacked['lever_position_fleetsize'].eq(1.0)&sf_stacked['lever_position_price'].eq(1.0))
                                                                                                  .groupby([sf_stacked['IDMerged'],sf_stacked['tripIndex']])
                                                                                                   .transform('first')
                                                                                                   .convert_dtypes())

CPU times: total: 3.08 s
Wall time: 3.19 s


<timed exec>:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`


In [14]:
sf_stacked['door_to_door_time_relative_to_baseline'] = sf_stacked['duration_door_to_door'] - (sf_stacked['duration_door_to_door'].where(sf_stacked['lever_position_fleetsize'].eq(1.0))
                                                                                                  .groupby([sf_stacked['IDMerged'],sf_stacked['tripIndex']])
                                                                                                   .transform('first')
                                                                                                   .convert_dtypes())

C:\Users\nazanin\AppData\Local\Temp\2\ipykernel_8860\1041979548.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sf_stacked['door_to_door_time_relative_to_baseline'] = sf_stacked['duration_door_to_door'] - (sf_stacked['duration_door_to_door'].where(sf_stacked['lever_position_fleetsize'].eq(1.0))


In [25]:
# shift columns to first position
first_column = sf_stacked.pop('logsum_relative_to_baseline')
second_column = sf_stacked.pop('door_to_door_time_relative_to_baseline')
third_column = sf_stacked.pop('income')
fourth_column = sf_stacked.pop('actPurpose')
# insert column using insert(position,column_name,first_column) function
sf_stacked.insert(5, 'logsum_relative_to_baseline', first_column)
sf_stacked.insert(6, 'door_to_door_time_relative_to_baseline', second_column)
sf_stacked.insert(7, 'income', third_column)
sf_stacked.insert(8, 'actPurpose', fourth_column)

In [26]:
# Drop unused columns
sf_stacked = sf_stacked.drop(['sf_detached','destination_logsum_x',
                                          'household_id_x','destination_logsum_y','logsum_tours_mode_AS_tours','primary_purpose_y',
                                          'PNUM','workplace_location_logsum','work_auto_savings','mandatory_tour_frequency','lever_position'], axis=1)

In [15]:
# Rename to potential INEXUS
sf_stacked.rename(columns={"logsum_trip_mode_AS_trips":"logsum_trip_potential_INEXUS"}, inplace=True) 

In [28]:
# shift columns to first position
#first_column = sf_stacked_tr_fr.pop('mode_choice_actual_6')
second_column = sf_stacked.pop('mode_choice_actual_BEAM')
#third_column = sf_stacked_tr_fr.pop('income')
#fourth_column = sf_stacked_tr_fr.pop('actPurpose')
# insert column using insert(position,column_name,first_column) function
#sf_stacked_tr_fr.insert(9, 'mode_choice_actual_6', first_column)
sf_stacked.insert(10, 'mode_choice_actual_BEAM', second_column)
#sf_stacked_tr_fr.insert(11, 'income', third_column)
#sf_stacked_tr_fr.insert(12, 'actPurpose', fourth_column)

In [29]:
sf_stacked[sf_stacked['mode_choice_actual_6'] == 'ride_hail_transit'].shape

(2061, 129)

In [30]:
sf_stacked[sf_stacked['replanning_status'] > 0].shape

(45730, 129)

In [16]:
# Add the mandatory category column
mandatory = ['work' , 'univ', 'school']
sf_stacked['mandatoryCat'] = np.where((sf_stacked.actEndType.isin(mandatory)) & (sf_stacked.actStartType.isin(mandatory)), 'from_M_to_M' , None)

C:\Users\nazanin\AppData\Local\Temp\2\ipykernel_8860\875273223.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sf_stacked['mandatoryCat'] = np.where((sf_stacked.actEndType.isin(mandatory)) & (sf_stacked.actStartType.isin(mandatory)), 'from_M_to_M' , None)


In [17]:
sf_stacked['mandatoryCat'] = np.where((sf_stacked.actEndType == 'Home') & (sf_stacked.actStartType.isin(mandatory)), 'from_H_to_M' , sf_stacked['mandatoryCat'])

In [18]:
sf_stacked['mandatoryCat'] = np.where((sf_stacked.actEndType.isin(mandatory)) & (sf_stacked.actStartType == "Home"), 'from_M_to_H' , sf_stacked['mandatoryCat'])

In [19]:
non_mandatory = ['othmaint' , 'othdiscr', 'escort', 'eatout', 'social', 'shopping', 'atwork']
sf_stacked['mandatoryCat'] = np.where((sf_stacked.actEndType.isin(non_mandatory)) & (sf_stacked.actStartType.isin(non_mandatory)), 'from_N_to_N' , sf_stacked['mandatoryCat'])

In [35]:
# shift columns to first position
first_column = sf_stacked.pop('mandatoryCat')
# insert column using insert(position,column_name,first_column) function
sf_stacked.insert(11, 'mandatoryCat', first_column)

In [36]:
# Drop unused columns
sf_stacked = sf_stacked.drop(['num_work_tours','num_joint_tours', 'non_mandatory_tour_frequency',
                                          'num_non_mand','num_escort_tours','num_eatout_tours','num_shop_tours',
                                          'num_maint_tours','num_discr_tours','num_social_tours','num_non_escort_tours'], axis=1)

In [37]:
sf_stacked.mode_choice_actual_BEAM.unique()

array(['hov2_teleportation', 'car', 'hov3_teleportation', 'walk',
       'car_hov2', 'walk_transit', 'drive_transit', 'bike', 'car_hov3',
       'ride_hail', 'ride_hail_pooled', 'ride_hail_transit',
       'bike_transit'], dtype=object)

In [38]:
sf_stacked.trip_mode_AS_trips.unique()

array(['SHARED2FREE', 'DRIVEALONEFREE', 'DRIVEALONEPAY', 'SHARED2PAY',
       'SHARED3PAY', 'SHARED3FREE', 'WALK', 'WALK_LOC', 'TNC_SINGLE',
       'TAXI', 'DRIVE_HVY', 'BIKE', 'TNC_SHARED', 'WALK_LRF', 'WALK_HVY',
       'WALK_COM', 'DRIVE_LOC', 'DRIVE_LRF', 'DRIVE_COM'], dtype=object)

In [23]:
sf_stacked.head()

,IDMerged,tripIndex,scenario,lever,lever_position_fleetsize,lever_position_price,logsum_trip_potential_INEXUS,BlockGroupEnd,BlockGroupStart,actEndTime,actEndType,actStartTime,actStartType,cost_BEAM,distance_bike,distance_mode_choice,distance_privateCar,distance_ridehail,distance_transit,distance_travelling,distance_walking,duration_in_privateCar,duration_in_ridehail,duration_in_transit,duration_on_bike,duration_travelling,duration_walking,emissionBiodiesel,emissionDiesel,emissionElectricity,emissionFood,emissionGasoline,emission_marginal,fuelBiodiesel,fuelDiesel,fuelElectricity,fuelFood,fuelGasoline,fuel_marginal,fuel_not_Food,vehicleIds_estimate,mode_choice_actual_BEAM,mode_choice_planned_BEAM,numPassengers,parkingType,reason,replanning_status,ride_hail_pooled,transit_bus,transit_cable_car,transit_rail,transit_subway,transit_tram,vehicleIds,year,duration_door_to_door,waitTime,actPurpose,mode_choice_actual_5,mode_choice_actual_6,trip_id,person_id,household_id,tour_id,primary_purpose_x,trip_num,outbound,trip_count,purpose,destination_x,origin_x,destination_logsum_x,depart,trip_mode_AS_trips,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,number_of_participants,destination_y,origin_y,household_id_x,tdd,start,end,duration,composition,destination_logsum_y,tour_mode_AS_tours,logsum_tours_mode_AS_tours,atwork_subtour_frequency,parent_tour_id,stop_frequency,primary_purpose_y,earning,worker,student,hispanic.1,person_sex,PNUM,sex,household_id_y,hours,MAR,person_age,edu,school_zone_id,race_id,race,work_at_home,age,work_zone_id,p_hispanic,relate,hispanic,TAZ_x,ptype,pemploy,pstudent,home_x,home_y,age_16_to_19,age_16_p,adult,male,female,has_non_worker,has_retiree,has_preschool_kid,has_driving_kid,has_school_kid,has_full_time,has_part_time,has_university,student_is_employed,nonstudent_to_school,is_student,is_gradeschool,is_highschool,is_university,school_segment,is_worker,home_taz,value_of_time,school_taz,distance_to_school,roundtrip_auto_time_to_school,workplace_taz,workplace_location_logsum,distance_to_work,workplace_in_cbd,work_taz_area_type,roundtrip_auto_time_to_work,work_auto_savings,work_auto_savings_ratio,free_parking_at_work,cdap_activity,travel_active,under16_not_at_school,has_preschool_kid_at_home,has_school_kid_at_home,mandatory_tour_frequency,work_and_school_and_worker,work_and_school_and_student,num_mand,num_work_tours,num_joint_tours,non_mandatory_tour_frequency,num_non_mand,num_escort_tours,num_eatout_tours,num_shop_tours,num_maint_tours,num_discr_tours,num_social_tours,num_non_escort_tours,gt55,seniors,VEHICL,lcm_county_id,hh_children,block_id,gt2,hispanic_head,age_of_head,race_of_head,tenure_mover,hh_size,sf_detached,tenure,hh_cars,income,hh_age_of_head,serialno,num_workers,hh_race_of_head,hh_income,recent_mover,hh_workers,hispanic_status_of_head,hh_seniors,hhsize,hh_type,TAZ_y,HHT,sample_rate,chunk_id,income_in_thousands,income_segment,median_value_of_time,hh_value_of_time,num_non_workers,num_drivers,num_adults,num_children,num_young_children,num_children_5_to_15,num_children_16_to_17,num_college_age,num_young_adults,non_family,family,home_is_urban,home_is_rural,auto_ownership,hh_work_auto_savings_ratio,num_under16_not_at_school,num_travel_active,num_travel_active_adults,num_travel_active_preschoolers,num_travel_active_children,num_travel_active_non_preschoolers,participates_in_jtf_model,joint_tour_frequency,num_hh_joint_tours,DRIVEALONEFREE,DRIVEALONEPAY,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,WALK,BIKE,WALK_LOC,WALK_LRF,WALK_EXP,WALK_HVY,WALK_COM,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED,lever_position,logsum_relative_to_baseline,door_to_door_time_relative_to_baseline,mandatoryCat
0,1,577.000000,ridehail,price_fleetsize,0.500000,0.500000,-1.964102,60855033053.000000,60855035091.000000,30020.000000,Home,30169.000000,othmaint,0.000000,0.000000,3879.974000,3879.974000,0.000000,0.000000,3879.974000,0.000000,149.000000,0.000000,0.000000,0.000000,149.000000,0

In [24]:
# Rename to Potential INEXUS
sf_stacked.rename(columns={"logsum_trip_potential_INEXUS":"logsum_trip_Potential_INEXUS"}, inplace=True) 

In [28]:
# Mapping BEAM and ASIM modes
conditions  = [(sf_stacked['mode_choice_actual_BEAM'] == 'walk')&(sf_stacked['trip_mode_AS_trips'] == 'WALK'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'bike')&(sf_stacked['trip_mode_AS_trips'] == 'BIKE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'hov3_teleportation')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED3FREE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'hov3_teleportation')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED3PAY'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car_hov3')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED3FREE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car_hov3')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED3PAY'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'hov2_teleportation')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED2FREE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'hov2_teleportation')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED2PAY'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car_hov2')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED2PAY'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car_hov2')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED2FREE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVEALONEFREE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVEALONEPAY'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'walk_transit')&(sf_stacked['trip_mode_AS_trips'] == 'WALK_LRF'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'walk_transit')&(sf_stacked['trip_mode_AS_trips'] == 'WALK_LOC'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'walk_transit')&(sf_stacked['trip_mode_AS_trips'] == 'WALK_EXP'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'walk_transit')&(sf_stacked['trip_mode_AS_trips'] == 'WALK_HVY'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'ride_hail')&(sf_stacked['trip_mode_AS_trips'] == 'TNC_SINGLE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'ride_hail')&(sf_stacked['trip_mode_AS_trips'] == 'TNC_SHARED'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'ride_hail')&(sf_stacked['trip_mode_AS_trips'] == 'TAXI'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'walk_transit')&(sf_stacked['trip_mode_AS_trips'] == 'WALK_COM'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'drive_transit')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_COM'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'drive_transit')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_LRF'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'drive_transit')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_LOC'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'drive_transit')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_EXP'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'drive_transit')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_HVY'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'hov3_teleportation')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED2FREE'),
               
               (sf_stacked['mode_choice_actual_BEAM'] == 'car_hov2')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED3FREE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car_hov2')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED3PAY'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car_hov3')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED2FREE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car_hov3')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED2PAY'),
               
               (sf_stacked['mode_choice_actual_BEAM'] == 'hov2_teleportation')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED3FREE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'hov2_teleportation')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED3PAY')]


choices = [sf_stacked['WALK'], sf_stacked['BIKE'], sf_stacked['SHARED3FREE'], sf_stacked['SHARED3PAY'],
           sf_stacked['SHARED3FREE'], sf_stacked['SHARED3PAY'], sf_stacked['SHARED2FREE'], 
           sf_stacked['SHARED2PAY'], sf_stacked['SHARED2PAY'], sf_stacked['SHARED2FREE'], 
           sf_stacked['DRIVEALONEFREE'], sf_stacked['DRIVEALONEPAY'], sf_stacked['WALK_LRF'], 
           sf_stacked['WALK_LOC'], sf_stacked['WALK_EXP'], sf_stacked['WALK_HVY'], sf_stacked['TNC_SINGLE'], 
           sf_stacked['TNC_SHARED'], sf_stacked['TAXI'], sf_stacked['WALK_COM'], 
           sf_stacked['DRIVE_COM'], sf_stacked['DRIVE_LRF'], sf_stacked['DRIVE_LOC'], sf_stacked['DRIVE_EXP'],
           sf_stacked['DRIVE_HVY'], sf_stacked['SHARED3FREE'],
           sf_stacked['SHARED2FREE'], sf_stacked['SHARED2PAY'], sf_stacked['SHARED3FREE'],  sf_stacked['SHARED3PAY'],
           sf_stacked['SHARED2FREE'], sf_stacked['SHARED2PAY']]

In [29]:
sf_stacked['Realized_INEXUS'] = np.select(conditions, choices, default=np.nan)

In [30]:
# shift columns to first position
first_column = sf_stacked.pop('Realized_INEXUS')
# insert column using insert(position,column_name,first_column) function
sf_stacked.insert(5, 'Realized_INEXUS', first_column)

C:\Users\nazanin\AppData\Local\Temp\2\ipykernel_8860\872035888.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sf_stacked.insert(5, 'Realized_INEXUS', first_column)


In [31]:
sf_stacked.head()

,IDMerged,tripIndex,scenario,lever,lever_position_fleetsize,Realized_INEXUS,lever_position_price,logsum_trip_Potential_INEXUS,BlockGroupEnd,BlockGroupStart,actEndTime,actEndType,actStartTime,actStartType,cost_BEAM,distance_bike,distance_mode_choice,distance_privateCar,distance_ridehail,distance_transit,distance_travelling,distance_walking,duration_in_privateCar,duration_in_ridehail,duration_in_transit,duration_on_bike,duration_travelling,duration_walking,emissionBiodiesel,emissionDiesel,emissionElectricity,emissionFood,emissionGasoline,emission_marginal,fuelBiodiesel,fuelDiesel,fuelElectricity,fuelFood,fuelGasoline,fuel_marginal,fuel_not_Food,vehicleIds_estimate,mode_choice_actual_BEAM,mode_choice_planned_BEAM,numPassengers,parkingType,reason,replanning_status,ride_hail_pooled,transit_bus,transit_cable_car,transit_rail,transit_subway,transit_tram,vehicleIds,year,duration_door_to_door,waitTime,actPurpose,mode_choice_actual_5,mode_choice_actual_6,trip_id,person_id,household_id,tour_id,primary_purpose_x,trip_num,outbound,trip_count,purpose,destination_x,origin_x,destination_logsum_x,depart,trip_mode_AS_trips,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,number_of_participants,destination_y,origin_y,household_id_x,tdd,start,end,duration,composition,destination_logsum_y,tour_mode_AS_tours,logsum_tours_mode_AS_tours,atwork_subtour_frequency,parent_tour_id,stop_frequency,primary_purpose_y,earning,worker,student,hispanic.1,person_sex,PNUM,sex,household_id_y,hours,MAR,person_age,edu,school_zone_id,race_id,race,work_at_home,age,work_zone_id,p_hispanic,relate,hispanic,TAZ_x,ptype,pemploy,pstudent,home_x,home_y,age_16_to_19,age_16_p,adult,male,female,has_non_worker,has_retiree,has_preschool_kid,has_driving_kid,has_school_kid,has_full_time,has_part_time,has_university,student_is_employed,nonstudent_to_school,is_student,is_gradeschool,is_highschool,is_university,school_segment,is_worker,home_taz,value_of_time,school_taz,distance_to_school,roundtrip_auto_time_to_school,workplace_taz,workplace_location_logsum,distance_to_work,workplace_in_cbd,work_taz_area_type,roundtrip_auto_time_to_work,work_auto_savings,work_auto_savings_ratio,free_parking_at_work,cdap_activity,travel_active,under16_not_at_school,has_preschool_kid_at_home,has_school_kid_at_home,mandatory_tour_frequency,work_and_school_and_worker,work_and_school_and_student,num_mand,num_work_tours,num_joint_tours,non_mandatory_tour_frequency,num_non_mand,num_escort_tours,num_eatout_tours,num_shop_tours,num_maint_tours,num_discr_tours,num_social_tours,num_non_escort_tours,gt55,seniors,VEHICL,lcm_county_id,hh_children,block_id,gt2,hispanic_head,age_of_head,race_of_head,tenure_mover,hh_size,sf_detached,tenure,hh_cars,income,hh_age_of_head,serialno,num_workers,hh_race_of_head,hh_income,recent_mover,hh_workers,hispanic_status_of_head,hh_seniors,hhsize,hh_type,TAZ_y,HHT,sample_rate,chunk_id,income_in_thousands,income_segment,median_value_of_time,hh_value_of_time,num_non_workers,num_drivers,num_adults,num_children,num_young_children,num_children_5_to_15,num_children_16_to_17,num_college_age,num_young_adults,non_family,family,home_is_urban,home_is_rural,auto_ownership,hh_work_auto_savings_ratio,num_under16_not_at_school,num_travel_active,num_travel_active_adults,num_travel_active_preschoolers,num_travel_active_children,num_travel_active_non_preschoolers,participates_in_jtf_model,joint_tour_frequency,num_hh_joint_tours,DRIVEALONEFREE,DRIVEALONEPAY,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,WALK,BIKE,WALK_LOC,WALK_LRF,WALK_EXP,WALK_HVY,WALK_COM,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED,lever_position,logsum_relative_to_baseline,door_to_door_time_relative_to_baseline,mandatoryCat
0,1,577.000000,ridehail,price_fleetsize,0.500000,-2.321091,0.500000,-1.964102,60855033053.000000,60855035091.000000,30020.000000,Home,30169.000000,othmaint,0.000000,0.000000,3879.974000,3879.974000,0.000000,0.000000,3879.974000,0.000000,149.000000,0.000000,0.00

In [32]:
sf_stacked[sf_stacked['Realized_INEXUS'].isna()].shape

(1899418, 263)

In [45]:
#sf_stacked[sf_stacked['Realized_INEXUS'].isna()].head(10000).to_csv('C:/Shared-Work/Data/CleanData/ASIM_BEAM_Merged/realizedIN_empty.csv', index = False)

In [46]:
#%%time
#sf_stacked_tr_fr.to_csv('s3://beam-core-act/deepDive/CleanData/SanFrancisco/Stacked/sf_2018_stacked_tr_fr_v3.csv', index=False)

In [33]:
sf_stacked = sf_stacked[sf_stacked['trip_mode_AS_trips'].notna()]

In [ ]:
sf_stacked = sf_stacked.drop(['Realized_INEXUS'], axis=1)

In [49]:
# Mapping BEAM and ASIM modes
conditions  = [(sf_stacked['mode_choice_actual_BEAM'] == 'walk'), 
               (sf_stacked['mode_choice_actual_BEAM'] == 'bike'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'hov3_teleportation')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED3FREE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'hov3_teleportation')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED3PAY'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car_hov3')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED3FREE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car_hov3')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED3PAY'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'hov2_teleportation')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED2FREE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'hov2_teleportation')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED2PAY'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car_hov2')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED2PAY'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car_hov2')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED2FREE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVEALONEFREE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVEALONEPAY'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'walk_transit')&(sf_stacked['trip_mode_AS_trips'] == 'WALK_LRF'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'walk_transit')&(sf_stacked['trip_mode_AS_trips'] == 'WALK_LOC'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'walk_transit')&(sf_stacked['trip_mode_AS_trips'] == 'WALK_EXP'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'walk_transit')&(sf_stacked['trip_mode_AS_trips'] == 'WALK_HVY'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'ride_hail')&(sf_stacked['trip_mode_AS_trips'] == 'TNC_SINGLE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'ride_hail')&(sf_stacked['trip_mode_AS_trips'] == 'TNC_SHARED'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'ride_hail')&(sf_stacked['trip_mode_AS_trips'] == 'TAXI'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'walk_transit')&(sf_stacked['trip_mode_AS_trips'] == 'WALK_COM'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'drive_transit')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_COM'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'drive_transit')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_LRF'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'drive_transit')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_LOC'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'drive_transit')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_EXP'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'drive_transit')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_HVY'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'hov3_teleportation')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED2FREE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED2PAY'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED3PAY'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car_hov2')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED3FREE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car_hov2')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED3PAY'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car_hov3')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED2FREE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car_hov3')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED2PAY'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'hov2_teleportation')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED3FREE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'hov2_teleportation')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED3PAY'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED3FREE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car_hov2')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVEALONEPAY'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car_hov2')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVEALONEFREE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'hov2_teleportation')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVEALONEFREE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'hov2_teleportation')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVEALONEPAY'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED2FREE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car_hov3')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVEALONEPAY'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car_hov3')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVEALONEFREE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'hov3_teleportation')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVEALONEFREE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'hov3_teleportation')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVEALONEPAY'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'hov3_teleportation')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED2PAY'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'ride_hail_pooled')&(sf_stacked['ride_hail_pooled'] > 1),
               (sf_stacked['mode_choice_actual_BEAM'] == 'hov3_teleportation')&(sf_stacked['trip_mode_AS_trips'] == 'WALK'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'hov2_teleportation')&(sf_stacked['trip_mode_AS_trips'] == 'WALK'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car_hov2')&((sf_stacked['trip_mode_AS_trips'] == 'WALK_HVY')|
                    (sf_stacked['trip_mode_AS_trips'] == 'WALK_COM')|(sf_stacked['trip_mode_AS_trips'] == 'WALK_EXP')|
                    (sf_stacked['trip_mode_AS_trips'] == 'DRIVE_COM')|(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_EXP')|
                    (sf_stacked['trip_mode_AS_trips'] == 'DRIVE_HVY')|(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_LOC')|
                    (sf_stacked['trip_mode_AS_trips'] == 'DRIVE_LRF')|(sf_stacked['trip_mode_AS_trips'] == 'WALK_LRF')| 
                    (sf_stacked['trip_mode_AS_trips'] == 'WALK_LOC')|(sf_stacked['trip_mode_AS_trips'] == 'WALK')|
                    (sf_stacked['trip_mode_AS_trips'] == 'BIKE')|(sf_stacked['trip_mode_AS_trips'] == 'TNC_SINGLE')|
                    (sf_stacked['trip_mode_AS_trips'] == 'TNC_SHARED')|(sf_stacked['trip_mode_AS_trips'] == 'TAXI'))
                    & (sf_stacked['cost_BEAM'] > 2.5),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car_hov3')&((sf_stacked['trip_mode_AS_trips'] == 'WALK_HVY')|
                    (sf_stacked['trip_mode_AS_trips'] == 'WALK_COM')|(sf_stacked['trip_mode_AS_trips'] == 'WALK_EXP')|
                    (sf_stacked['trip_mode_AS_trips'] == 'DRIVE_COM')|(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_EXP')|
                    (sf_stacked['trip_mode_AS_trips'] == 'DRIVE_HVY')|(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_LOC')|
                    (sf_stacked['trip_mode_AS_trips'] == 'DRIVE_LRF')|(sf_stacked['trip_mode_AS_trips'] == 'WALK_LRF')| 
                    (sf_stacked['trip_mode_AS_trips'] == 'WALK_LOC')|(sf_stacked['trip_mode_AS_trips'] == 'WALK')|
                    (sf_stacked['trip_mode_AS_trips'] == 'BIKE')|(sf_stacked['trip_mode_AS_trips'] == 'TNC_SINGLE')|
                    (sf_stacked['trip_mode_AS_trips'] == 'TNC_SHARED')|(sf_stacked['trip_mode_AS_trips'] == 'TAXI'))
                    & (sf_stacked['cost_BEAM'] > 2.5),
              (sf_stacked['mode_choice_actual_BEAM'] == 'car_hov2')&((sf_stacked['trip_mode_AS_trips'] == 'WALK_HVY')|
                    (sf_stacked['trip_mode_AS_trips'] == 'WALK_COM')|(sf_stacked['trip_mode_AS_trips'] == 'WALK_EXP')|
                    (sf_stacked['trip_mode_AS_trips'] == 'DRIVE_COM')|(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_EXP')|
                    (sf_stacked['trip_mode_AS_trips'] == 'DRIVE_HVY')|(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_LOC')|
                    (sf_stacked['trip_mode_AS_trips'] == 'DRIVE_LRF')|(sf_stacked['trip_mode_AS_trips'] == 'WALK_LRF')| 
                    (sf_stacked['trip_mode_AS_trips'] == 'WALK_LOC')|(sf_stacked['trip_mode_AS_trips'] == 'WALK')|
                    (sf_stacked['trip_mode_AS_trips'] == 'BIKE')|(sf_stacked['trip_mode_AS_trips'] == 'TNC_SINGLE')|
                    (sf_stacked['trip_mode_AS_trips'] == 'TNC_SHARED')|(sf_stacked['trip_mode_AS_trips'] == 'TAXI'))
                    & (sf_stacked['cost_BEAM'] <= 2.5),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car_hov3')&((sf_stacked['trip_mode_AS_trips'] == 'WALK_HVY')|
                    (sf_stacked['trip_mode_AS_trips'] == 'WALK_COM')|(sf_stacked['trip_mode_AS_trips'] == 'WALK_EXP')|
                    (sf_stacked['trip_mode_AS_trips'] == 'DRIVE_COM')|(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_EXP')|
                    (sf_stacked['trip_mode_AS_trips'] == 'DRIVE_HVY')|(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_LOC')|
                    (sf_stacked['trip_mode_AS_trips'] == 'DRIVE_LRF')|(sf_stacked['trip_mode_AS_trips'] == 'WALK_LRF')| 
                    (sf_stacked['trip_mode_AS_trips'] == 'WALK_LOC')|(sf_stacked['trip_mode_AS_trips'] == 'WALK')|
                    (sf_stacked['trip_mode_AS_trips'] == 'BIKE')|(sf_stacked['trip_mode_AS_trips'] == 'TNC_SINGLE')|
                    (sf_stacked['trip_mode_AS_trips'] == 'TNC_SHARED')|(sf_stacked['trip_mode_AS_trips'] == 'TAXI'))
                    & (sf_stacked['cost_BEAM'] <= 2.5),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car')&((sf_stacked['trip_mode_AS_trips'] == 'WALK_HVY')|
                    (sf_stacked['trip_mode_AS_trips'] == 'WALK_COM')|(sf_stacked['trip_mode_AS_trips'] == 'WALK_EXP')|
                    (sf_stacked['trip_mode_AS_trips'] == 'DRIVE_COM')|(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_EXP')|
                    (sf_stacked['trip_mode_AS_trips'] == 'DRIVE_HVY')|(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_LOC')|
                    (sf_stacked['trip_mode_AS_trips'] == 'DRIVE_LRF')|(sf_stacked['trip_mode_AS_trips'] == 'WALK_LRF')| 
                    (sf_stacked['trip_mode_AS_trips'] == 'WALK_LOC')|(sf_stacked['trip_mode_AS_trips'] == 'WALK')|
                    (sf_stacked['trip_mode_AS_trips'] == 'BIKE')|(sf_stacked['trip_mode_AS_trips'] == 'TNC_SINGLE')|
                    (sf_stacked['trip_mode_AS_trips'] == 'TNC_SHARED')|(sf_stacked['trip_mode_AS_trips'] == 'TAXI'))
                    & (sf_stacked['cost_BEAM'] <= 2.5),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car')&((sf_stacked['trip_mode_AS_trips'] == 'WALK_HVY')|
                    (sf_stacked['trip_mode_AS_trips'] == 'WALK_COM')|(sf_stacked['trip_mode_AS_trips'] == 'WALK_EXP')|
                    (sf_stacked['trip_mode_AS_trips'] == 'DRIVE_COM')|(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_EXP')|
                    (sf_stacked['trip_mode_AS_trips'] == 'DRIVE_HVY')|(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_LOC')|
                    (sf_stacked['trip_mode_AS_trips'] == 'DRIVE_LRF')|(sf_stacked['trip_mode_AS_trips'] == 'WALK_LRF')| 
                    (sf_stacked['trip_mode_AS_trips'] == 'WALK_LOC')|(sf_stacked['trip_mode_AS_trips'] == 'WALK')|
                    (sf_stacked['trip_mode_AS_trips'] == 'BIKE')|(sf_stacked['trip_mode_AS_trips'] == 'TNC_SINGLE')|
                    (sf_stacked['trip_mode_AS_trips'] == 'TNC_SHARED')|(sf_stacked['trip_mode_AS_trips'] == 'TAXI'))
                    & (sf_stacked['cost_BEAM'] > 2.5),
               (sf_stacked['mode_choice_actual_BEAM'] == 'ride_hail_pooled'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'ride_hail_transit')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_COM'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'ride_hail_transit')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_LRF'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'ride_hail_transit')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_LOC'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'ride_hail_transit')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_EXP'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'ride_hail_transit')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_HVY'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'ride_hail_transit')&(sf_stacked['trip_mode_AS_trips'] == 'WALK_COM'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'ride_hail_transit')&(sf_stacked['trip_mode_AS_trips'] == 'WALK_LRF'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'ride_hail_transit')&(sf_stacked['trip_mode_AS_trips'] == 'WALK_LOC'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'ride_hail_transit')&(sf_stacked['trip_mode_AS_trips'] == 'WALK_EXP'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'ride_hail_transit')&(sf_stacked['trip_mode_AS_trips'] == 'WALK_HVY')]
        
choices = [sf_stacked['WALK'], sf_stacked['BIKE'], sf_stacked['SHARED3FREE'], sf_stacked['SHARED3PAY'],
           sf_stacked['SHARED3FREE'], sf_stacked['SHARED3PAY'], sf_stacked['SHARED2FREE'], 
           sf_stacked['SHARED2PAY'], sf_stacked['SHARED2PAY'], sf_stacked['SHARED2FREE'], 
           sf_stacked['DRIVEALONEFREE'], sf_stacked['DRIVEALONEPAY'], sf_stacked['WALK_LRF'], 
           sf_stacked['WALK_LOC'], sf_stacked['WALK_EXP'], sf_stacked['WALK_HVY'], sf_stacked['TNC_SINGLE'], 
           sf_stacked['TNC_SHARED'], sf_stacked['TAXI'], sf_stacked['WALK_COM'], 
           sf_stacked['DRIVE_COM'], sf_stacked['DRIVE_LRF'], sf_stacked['DRIVE_LOC'], sf_stacked['DRIVE_EXP'],
           sf_stacked['DRIVE_HVY'], sf_stacked['SHARED3FREE'], sf_stacked['DRIVEALONEPAY'], sf_stacked['DRIVEALONEPAY'],
           sf_stacked['SHARED2FREE'], sf_stacked['SHARED2PAY'], sf_stacked['SHARED3FREE'],  sf_stacked['SHARED3PAY'],
           sf_stacked['SHARED2FREE'], sf_stacked['SHARED2PAY'], sf_stacked['DRIVEALONEFREE'], sf_stacked['SHARED2PAY'], 
           sf_stacked['SHARED2FREE'], sf_stacked['SHARED2FREE'], sf_stacked['SHARED2PAY'], sf_stacked['DRIVEALONEFREE'], 
           sf_stacked['SHARED3PAY'], sf_stacked['SHARED3FREE'], sf_stacked['SHARED3FREE'], sf_stacked['SHARED3PAY'],
           sf_stacked['SHARED3PAY'], sf_stacked['TNC_SHARED'], sf_stacked['SHARED3FREE'], sf_stacked['SHARED2FREE'],
           sf_stacked['SHARED2PAY'], sf_stacked['SHARED3PAY'], sf_stacked['SHARED2FREE'],sf_stacked['SHARED3FREE'],
           sf_stacked['DRIVEALONEFREE'], sf_stacked['DRIVEALONEPAY'], sf_stacked['TNC_SHARED'], sf_stacked['DRIVE_COM'],
           sf_stacked['DRIVE_LRF'], sf_stacked['DRIVE_LOC'], sf_stacked['DRIVE_EXP'], sf_stacked['DRIVE_HVY'],
           sf_stacked['WALK_COM'], sf_stacked['WALK_LRF'], sf_stacked['WALK_LOC'], sf_stacked['WALK_EXP'],
           sf_stacked['WALK_HVY']]

In [50]:
sf_stacked['Realized_INEXUS'] = np.select(conditions, choices, default=np.nan)

In [51]:
# shift columns to first position
first_column = sf_stacked.pop('Realized_INEXUS')
# insert column using insert(position,column_name,first_column) function
sf_stacked.insert(5, 'Realized_INEXUS', first_column)

In [52]:
sf_stacked['Realized_INEXUS'] = np.where((sf_stacked['Realized_INEXUS'].isna())&
                                               (sf_stacked['mode_choice_actual_BEAM'] == 'ride_hail')&
                                               ((sf_stacked['numPassengers'] == '1.0, 1.0, 1.0')|
                                                (sf_stacked['numPassengers'] =='1.0, 1.0, 1.0, 1.0')|
                                                (sf_stacked['numPassengers'] =='1.0, 1.0, 1.0, 1.0, 1.0')|
                                                (sf_stacked['numPassengers'] =='1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0')|
                                                (sf_stacked['numPassengers'] =='1.0, 8.0, 8.0, 7.0, 1.0, 1.0, 1.0, 1.0')|
                                                (sf_stacked['numPassengers'] =='1.0, 16.0, 1.0, 1.0, 1.0, 1.0')|
                                                (sf_stacked['numPassengers'] =='1.0, 10.0, 11.0, 1.0, 1.0, 1.0, 1.0')), 
                                                sf_stacked['TNC_SINGLE'], sf_stacked['Realized_INEXUS']) 

In [53]:
sf_stacked['Realized_INEXUS'] = np.where((sf_stacked['Realized_INEXUS'].isna())&
                                               (sf_stacked['mode_choice_actual_BEAM'] == 'hov2_teleportation'), 
                                                sf_stacked.loc[:, ["SHARED2FREE", "SHARED2PAY"]].max(1), sf_stacked['Realized_INEXUS']) 

In [54]:
sf_stacked['Realized_INEXUS'] = np.where((sf_stacked['Realized_INEXUS'].isna())&
                                               (sf_stacked['mode_choice_actual_BEAM'] == 'hov3_teleportation'), 
                                                sf_stacked.loc[:, ["SHARED3FREE", "SHARED3PAY"]].max(1), sf_stacked['Realized_INEXUS']) 

In [55]:
sf_stacked['Realized_INEXUS'] = np.where((sf_stacked['Realized_INEXUS'].isna())&
                                               (sf_stacked['mode_choice_actual_BEAM'] == 'walk_transit'), 
                                                sf_stacked.loc[:, ["WALK_COM", "WALK_LRF", "WALK_LOC", "WALK_EXP",
                                                      "WALK_HVY"]].max(1), sf_stacked['Realized_INEXUS']) 

In [56]:
sf_stacked['Realized_INEXUS'] = np.where((sf_stacked['Realized_INEXUS'].isna())&
                                               (sf_stacked['mode_choice_actual_BEAM'] == 'drive_transit'), 
                                                sf_stacked.loc[:, ["DRIVE_COM", "DRIVE_LRF", "DRIVE_LOC", "DRIVE_EXP",
                                                      "DRIVE_HVY"]].max(1), sf_stacked['Realized_INEXUS']) 

In [57]:
sf_stacked['Realized_INEXUS'] = np.where((sf_stacked['Realized_INEXUS'].isna())&
                                               (sf_stacked['mode_choice_actual_BEAM'] == 'bike_transit')&
                                               (sf_stacked['trip_mode_AS_trips'] == 'BIKE'), 
                                                sf_stacked['BIKE'], sf_stacked['Realized_INEXUS']) 

In [58]:
sf_stacked['Realized_INEXUS'] = np.where((sf_stacked['Realized_INEXUS'].isna())&
                                               (sf_stacked['mode_choice_actual_BEAM'] == 'bike_transit')&
                                               (sf_stacked['trip_mode_AS_trips'] == 'BIKE'), 
                                                sf_stacked['BIKE'], sf_stacked['Realized_INEXUS']) 

In [59]:
sf_stacked['Realized_INEXUS'] = np.where((sf_stacked['Realized_INEXUS'].isna())&
                                               (sf_stacked['mode_choice_actual_BEAM'] == 'ride_hail'), 
                                                sf_stacked['TNC_SINGLE'], sf_stacked['Realized_INEXUS'])

In [60]:
sf_stacked[sf_stacked['Realized_INEXUS'].isna()].shape

(704, 120)

In [61]:
#sf_stacked_tr_fr['Realized_INEXUS'] = np.where((sf_stacked_tr_fr['Realized_INEXUS'].isna())&
                                               #(sf_stacked_tr_fr['mode_choice_actual_BEAM'] == 'walk_transit')&
                                               #(sf_stacked_tr_fr['vehicleIds'].str.contains('(caltrain)')), 
                                               #sf_stacked_tr_fr['WALK_COM'], sf_stacked_tr_fr['Realized_INEXUS'])

In [62]:
sf_stacked = sf_stacked[sf_stacked['Realized_INEXUS'].notna()]

In [63]:
#sf_stacked_tr_fr[sf_stacked_tr_fr['Realized_INEXUS'].isna()].to_csv('C:/Shared-Work/Data/CleanData/ASIM_BEAM_Merged/realizedIN_empty.csv', index = False)

#### Social INEXUS

In [34]:
sf_stacked.income.quantile([0, 0.25,0.5,0.75, 1.00])

0.000000    -14600.000000
0.250000     50000.000000
0.500000     94000.000000
0.750000    155000.000000
1.000000   1354000.000000
Name: income, dtype: float64

In [35]:
# determine the quartile values to use with pd.cut
quartiles = sf_stacked['income'].quantile([.25, .5, .75]).tolist()

In [36]:
# add a lower and upper range for the bins in pd.cut
quartiles = [0] + quartiles + [float('inf')]

In [37]:
# add a quartiles column to groups, using pd.cut
sf_stacked['income_quartiles'] = pd.cut(sf_stacked['income'], bins=quartiles, labels=['1stQ', '2ndQ', '3rdQ', '4thQ'])

In [39]:
# determine the quartile values to use with pd.cut
deciles = sf_stacked['income'].quantile([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]).tolist()

In [40]:
# Column with six summarized modes
conditions  = [(sf_stacked['income'] >= -14600) & (sf_stacked['income'] < 24700), 
               (sf_stacked['income'] >= 24700) & (sf_stacked['income'] < 41240),
               (sf_stacked['income'] >=  41240) & (sf_stacked['income'] < 58000),
               (sf_stacked['income'] >= 58000) & (sf_stacked['income'] < 75000), 
               (sf_stacked['income'] >=  75000) & (sf_stacked['income'] < 94000),
               (sf_stacked['income'] >=  94000) & (sf_stacked['income'] < 114200),
               (sf_stacked['income'] >=  114200) & (sf_stacked['income'] < 140000),
               (sf_stacked['income'] >=  140000) & (sf_stacked['income'] < 175000),
               (sf_stacked['income'] >=  175000) & (sf_stacked['income'] < 241000),
               (sf_stacked['income'] >=  241000) & (sf_stacked['income'] <= 1354000)]

choices = [ '1stD', '2ndD', '3rdD', 
           '4thD', '5thD', '6thD', '7thD', '8thD', '9thD','10thD']

In [42]:
sf_stacked['income_deciles'] = np.select(conditions, choices, default=None)

In [69]:
m = sf_stacked.groupby(['income_quartiles'])['income'].median()

In [70]:
m

income_quartiles
1stQ    30000.000000
2ndQ    70600.000000
3rdQ   120000.000000
4thQ   219000.000000
Name: income, dtype: float64

In [43]:
# Mapping BEAM and ASIM modes
conditions  = [(sf_stacked['income_quartiles'] == '1stQ'), 
               (sf_stacked['income_quartiles'] == '2ndQ'),
               (sf_stacked['income_quartiles'] == '3rdQ'),
               (sf_stacked['income_quartiles'] == '4thQ')]

choices = [29300, 70000, 120000, 219000]

In [72]:
sf_stacked['MedianQuartiles'] = np.select(conditions, choices, default=np.nan)

In [45]:
sf_stacked['alpha'] = 0.1

In [47]:
sf_stacked['socialCarbonCost'] = sf_stacked['emission_marginal']*185 # or 102

In [48]:
sf_stacked['Social_INEXUS'] = (sf_stacked['Realized_INEXUS']/sf_stacked['alpha']) - sf_stacked['socialCarbonCost']

In [49]:
sf_stacked['Potential_INEXUS_in_dollar'] = sf_stacked['logsum_trip_Potential_INEXUS']/sf_stacked['alpha']

In [50]:
sf_stacked['Realized_INEXUS_in_dollar'] = sf_stacked['Realized_INEXUS']/sf_stacked['alpha']

#### The following chunks are useful but should be checked in the next runs (they resulted in exact same realized and potential inexus in dollar

In [78]:
# shift columns to first position
#first_column = sf_stacked.pop('Social_INEXUS')
# insert column using insert(position,column_name,first_column) function
#sf_stacked.insert(6, 'Social_INEXUS', first_column)

In [79]:
# shift columns to first position
#first_column = sf_stacked.pop('Potential_INEXUS_in_dollar')
#second_column = sf_stacked.pop('Realized_INEXUS_in_dollar')
# insert column using insert(position,column_name,first_column) function
#sf_stacked.insert(7, 'Potential_INEXUS_in_dollar', first_column)
#sf_stacked.insert(8, 'Realized_INEXUS_in_dollar', first_column)

In [51]:
sf_stacked.head()

,IDMerged,tripIndex,scenario,lever,lever_position_fleetsize,Realized_INEXUS,lever_position_price,logsum_trip_Potential_INEXUS,BlockGroupEnd,BlockGroupStart,actEndTime,actEndType,actStartTime,actStartType,cost_BEAM,distance_bike,distance_mode_choice,distance_privateCar,distance_ridehail,distance_transit,distance_travelling,distance_walking,duration_in_privateCar,duration_in_ridehail,duration_in_transit,duration_on_bike,duration_travelling,duration_walking,emissionBiodiesel,emissionDiesel,emissionElectricity,emissionFood,emissionGasoline,emission_marginal,fuelBiodiesel,fuelDiesel,fuelElectricity,fuelFood,fuelGasoline,fuel_marginal,fuel_not_Food,vehicleIds_estimate,mode_choice_actual_BEAM,mode_choice_planned_BEAM,numPassengers,parkingType,reason,replanning_status,ride_hail_pooled,transit_bus,transit_cable_car,transit_rail,transit_subway,transit_tram,vehicleIds,year,duration_door_to_door,waitTime,actPurpose,mode_choice_actual_5,mode_choice_actual_6,trip_id,person_id,household_id,tour_id,primary_purpose_x,trip_num,outbound,trip_count,purpose,destination_x,origin_x,destination_logsum_x,depart,trip_mode_AS_trips,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,number_of_participants,destination_y,origin_y,household_id_x,tdd,start,end,duration,composition,destination_logsum_y,tour_mode_AS_tours,logsum_tours_mode_AS_tours,atwork_subtour_frequency,parent_tour_id,stop_frequency,primary_purpose_y,earning,worker,student,hispanic.1,person_sex,PNUM,sex,household_id_y,hours,MAR,person_age,edu,school_zone_id,race_id,race,work_at_home,age,work_zone_id,p_hispanic,relate,hispanic,TAZ_x,ptype,pemploy,pstudent,home_x,home_y,age_16_to_19,age_16_p,adult,male,female,has_non_worker,has_retiree,has_preschool_kid,has_driving_kid,has_school_kid,has_full_time,has_part_time,has_university,student_is_employed,nonstudent_to_school,is_student,is_gradeschool,is_highschool,is_university,school_segment,is_worker,home_taz,value_of_time,school_taz,distance_to_school,roundtrip_auto_time_to_school,workplace_taz,workplace_location_logsum,distance_to_work,workplace_in_cbd,work_taz_area_type,roundtrip_auto_time_to_work,work_auto_savings,work_auto_savings_ratio,free_parking_at_work,cdap_activity,travel_active,under16_not_at_school,has_preschool_kid_at_home,has_school_kid_at_home,mandatory_tour_frequency,work_and_school_and_worker,work_and_school_and_student,num_mand,num_work_tours,num_joint_tours,non_mandatory_tour_frequency,num_non_mand,num_escort_tours,num_eatout_tours,num_shop_tours,num_maint_tours,num_discr_tours,num_social_tours,num_non_escort_tours,gt55,seniors,VEHICL,lcm_county_id,hh_children,block_id,gt2,hispanic_head,age_of_head,race_of_head,tenure_mover,hh_size,sf_detached,tenure,hh_cars,income,hh_age_of_head,serialno,num_workers,hh_race_of_head,hh_income,recent_mover,hh_workers,hispanic_status_of_head,hh_seniors,hhsize,hh_type,TAZ_y,HHT,sample_rate,chunk_id,income_in_thousands,income_segment,median_value_of_time,hh_value_of_time,num_non_workers,num_drivers,num_adults,num_children,num_young_children,num_children_5_to_15,num_children_16_to_17,num_college_age,num_young_adults,non_family,family,home_is_urban,home_is_rural,auto_ownership,hh_work_auto_savings_ratio,num_under16_not_at_school,num_travel_active,num_travel_active_adults,num_travel_active_preschoolers,num_travel_active_children,num_travel_active_non_preschoolers,participates_in_jtf_model,joint_tour_frequency,num_hh_joint_tours,DRIVEALONEFREE,DRIVEALONEPAY,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,WALK,BIKE,WALK_LOC,WALK_LRF,WALK_EXP,WALK_HVY,WALK_COM,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED,lever_position,logsum_relative_to_baseline,door_to_door_time_relative_to_baseline,mandatoryCat,income_quartiles,income_deciles,alpha,socialCarbonCost,Social_INEXUS,Potential_INEXUS_in_dollar,Realized_INEXUS_in_dollar
0,1,577.000000,ridehail,price_fleetsize,0.500000,-2.321091,0.500000,-1.964102,60855033053.000000,60855035091.000000,30020.000000,Home,30169

In [52]:
%%time
sf_stacked.to_csv('s3://beam-core-act/deepDive/CleanData/SanFrancisco/Stacked/Temp_AllCols/sf_2018_stacked_rh_price_fleetsz_0p5.csv', index=False)

CPU times: total: 20min 22s
Wall time: 27min 25s


In [53]:
sf_stacked.head()

,IDMerged,tripIndex,scenario,lever,lever_position_fleetsize,Realized_INEXUS,lever_position_price,logsum_trip_Potential_INEXUS,BlockGroupEnd,BlockGroupStart,actEndTime,actEndType,actStartTime,actStartType,cost_BEAM,distance_bike,distance_mode_choice,distance_privateCar,distance_ridehail,distance_transit,distance_travelling,distance_walking,duration_in_privateCar,duration_in_ridehail,duration_in_transit,duration_on_bike,duration_travelling,duration_walking,emissionBiodiesel,emissionDiesel,emissionElectricity,emissionFood,emissionGasoline,emission_marginal,fuelBiodiesel,fuelDiesel,fuelElectricity,fuelFood,fuelGasoline,fuel_marginal,fuel_not_Food,vehicleIds_estimate,mode_choice_actual_BEAM,mode_choice_planned_BEAM,numPassengers,parkingType,reason,replanning_status,ride_hail_pooled,transit_bus,transit_cable_car,transit_rail,transit_subway,transit_tram,vehicleIds,year,duration_door_to_door,waitTime,actPurpose,mode_choice_actual_5,mode_choice_actual_6,trip_id,person_id,household_id,tour_id,primary_purpose_x,trip_num,outbound,trip_count,purpose,destination_x,origin_x,destination_logsum_x,depart,trip_mode_AS_trips,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,number_of_participants,destination_y,origin_y,household_id_x,tdd,start,end,duration,composition,destination_logsum_y,tour_mode_AS_tours,logsum_tours_mode_AS_tours,atwork_subtour_frequency,parent_tour_id,stop_frequency,primary_purpose_y,earning,worker,student,hispanic.1,person_sex,PNUM,sex,household_id_y,hours,MAR,person_age,edu,school_zone_id,race_id,race,work_at_home,age,work_zone_id,p_hispanic,relate,hispanic,TAZ_x,ptype,pemploy,pstudent,home_x,home_y,age_16_to_19,age_16_p,adult,male,female,has_non_worker,has_retiree,has_preschool_kid,has_driving_kid,has_school_kid,has_full_time,has_part_time,has_university,student_is_employed,nonstudent_to_school,is_student,is_gradeschool,is_highschool,is_university,school_segment,is_worker,home_taz,value_of_time,school_taz,distance_to_school,roundtrip_auto_time_to_school,workplace_taz,workplace_location_logsum,distance_to_work,workplace_in_cbd,work_taz_area_type,roundtrip_auto_time_to_work,work_auto_savings,work_auto_savings_ratio,free_parking_at_work,cdap_activity,travel_active,under16_not_at_school,has_preschool_kid_at_home,has_school_kid_at_home,mandatory_tour_frequency,work_and_school_and_worker,work_and_school_and_student,num_mand,num_work_tours,num_joint_tours,non_mandatory_tour_frequency,num_non_mand,num_escort_tours,num_eatout_tours,num_shop_tours,num_maint_tours,num_discr_tours,num_social_tours,num_non_escort_tours,gt55,seniors,VEHICL,lcm_county_id,hh_children,block_id,gt2,hispanic_head,age_of_head,race_of_head,tenure_mover,hh_size,sf_detached,tenure,hh_cars,income,hh_age_of_head,serialno,num_workers,hh_race_of_head,hh_income,recent_mover,hh_workers,hispanic_status_of_head,hh_seniors,hhsize,hh_type,TAZ_y,HHT,sample_rate,chunk_id,income_in_thousands,income_segment,median_value_of_time,hh_value_of_time,num_non_workers,num_drivers,num_adults,num_children,num_young_children,num_children_5_to_15,num_children_16_to_17,num_college_age,num_young_adults,non_family,family,home_is_urban,home_is_rural,auto_ownership,hh_work_auto_savings_ratio,num_under16_not_at_school,num_travel_active,num_travel_active_adults,num_travel_active_preschoolers,num_travel_active_children,num_travel_active_non_preschoolers,participates_in_jtf_model,joint_tour_frequency,num_hh_joint_tours,DRIVEALONEFREE,DRIVEALONEPAY,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,WALK,BIKE,WALK_LOC,WALK_LRF,WALK_EXP,WALK_HVY,WALK_COM,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED,lever_position,logsum_relative_to_baseline,door_to_door_time_relative_to_baseline,mandatoryCat,income_quartiles,income_deciles,alpha,socialCarbonCost,Social_INEXUS,Potential_INEXUS_in_dollar,Realized_INEXUS_in_dollar
0,1,577.000000,ridehail,price_fleetsize,0.500000,-2.321091,0.500000,-1.964102,60855033053.000000,60855035091.000000,30020.000000,Home,30169